In [ ]:
import networkx as nx
import os
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import seaborn as sns
import matplotlib.patches as patches
import matplotlib.font_manager
from scipy.stats import pearsonr
from scipy.stats import linregress
from matplotlib import pyplot as plt
import matplotlib as mpl
from pycirclize import Circos
matplotlib.font_manager.fontManager.addfont('/h/tianyi/TS_datasets_reversion/Cell_Press_plot/Arial.ttf')
mpl.rcParams['pdf.fonttype'] = 42
mpl.rcParams['ps.fonttype'] = 42
plt.rcParams['font.family'] = 'Arial'
mpl.rcParams['font.size'] = 8 
plt.rcParams['axes.linewidth'] = 0.5
plt.rcParams['text.color'] = 'black'
plt.rcParams['axes.labelcolor'] = 'black'
plt.rcParams['xtick.color'] = 'black'
plt.rcParams['ytick.color'] = 'black'
plt.rcParams['axes.titlecolor'] = 'black'
plt.rcParams['legend.labelcolor'] = 'black'
plt.rcParams['axes.linewidth'] = 0.5

In [ ]:
## ROC of high-throughput drug screening experiments

import numpy
import networkx
import networkx as nx
from sklearn.metrics import roc_curve
from sklearn.metrics import roc_auc_score,precision_recall_curve,auc


cell_viability_distance_1um=pd.read_csv('/h/tianyi/TS_datasets_reversion/primary/predict_drug/high_throughput_drug_screening/CCK8_OD_1um_process/all_drug_score/cell_viability_distance_v5.txt',sep='\t')
all_hsa_score=pd.read_csv('/h/tianyi/TS_datasets_reversion/graduate_plot/result/drug_combination_cell_viability_corelation_network_distance/all_drug_score/distance_hsa_score_df_all.txt',sep='\t')
cell_viability_distance=cell_viability_distance_1um
drug_concentration='1μm'

cancer_drug_target=pd.read_excel('/h/tianyi/TS_datasets_reversion/drug_information/all_process/merge_info/screen_platform_library/approved_cancer_screen_library.xlsx')
drugbank_target=pd.read_csv('/h/tianyi/TS_datasets_reversion/drug_information/DGIdb/TOP_target/dgidb_score_cutoff_15.txt',sep='\t')
dgidb_score=pd.read_csv('/h/tianyi/TS_datasets_reversion/drug_information/DGIdb/DGIdb_json_score_drugbankid.txt',sep='\t')
dgidb_score_max=dgidb_score[['DrugBank','score']]
dgidb_score_max=dgidb_score_max.groupby('DrugBank').max()
dgidb_score_max['DrugBank']=dgidb_score_max.index.tolist()
dgidb_score_max.reset_index(drop=True,inplace=True)
dgidb_score_max.columns=['dgidb_score','DrugBank']
def core_convert_drug(method_score):
    drugbank_target_method_gene=pd.merge(method_score,drugbank_target,left_on='gene',right_on='target')
    cancer_drug_target_gene=pd.merge(drugbank_target_method_gene,cancer_drug_target,on='DrugBank')
    cancer_drug_target_gene_max=cancer_drug_target_gene.iloc[:,[1,4]]
    cancer_drug_target_gene_max.columns=['method_score','DrugBank']
    cancer_drug_target_gene_max=cancer_drug_target_gene_max.groupby('DrugBank').max()
    cancer_drug_target_gene_max['DrugBank']=cancer_drug_target_gene_max.index.tolist()
    cancer_drug_target_gene_max.reset_index(drop=True,inplace=True)
    return(cancer_drug_target_gene_max)


def network_distance_rank_label_prc(cell_viability_distance):
    top_gene_drug_top_num_label_sort=cell_viability_distance.sort_values('closest_distance_score')[::-1]
    top_gene_drug_top_num_label_sort['rank_score']=np.arange(1,top_gene_drug_top_num_label_sort.shape[0]+1)
    top_gene_drug_top_num_label_sort['label']=1
    top_gene_drug_top_num_label_sort.loc[top_gene_drug_top_num_label_sort['HSA_score']<0.15,'label']=0
    return(top_gene_drug_top_num_label_sort)

def other_method_plot_data_v2(betweenness_centrality_result):
    drugbank_target_method_gene=pd.merge(betweenness_centrality_result,drugbank_target,left_on='gene',right_on='target')
    betweenness_centrality_result_drug_rank=pd.merge(drugbank_target_method_gene,cancer_drug_target,on='DrugBank').iloc[:,[1,4]]
    betweenness_centrality_result_drug_rank.columns=['method_score','DrugBank']
    betweenness_centrality_result_prc_all=pd.merge(all_hsa_score,betweenness_centrality_result_drug_rank,left_on='drugbank_id',right_on='DrugBank')
    betweenness_centrality_result_prc_all=betweenness_centrality_result_prc_all[['drugbank_id','hsa_score','method_score']]
    betweenness_centrality_result_prc_all=betweenness_centrality_result_prc_all.drop_duplicates(['drugbank_id']).sort_values('method_score')[::-1]
    betweenness_centrality_result_prc_all.reset_index(drop=True,inplace=True)
    betweenness_centrality_result_prc_all['label']=1
    betweenness_centrality_result_prc_all.loc[betweenness_centrality_result_prc_all['hsa_score']<0.15,'label']=0
    betweenness_centrality_result_prc_all=betweenness_centrality_result_prc_all.dropna()
    
    betweenness_centrality_result_prc_all['rank_score']=list(range(1,betweenness_centrality_result_prc_all.shape[0]+1))
    betweenness_score = np.array(betweenness_centrality_result_prc_all['rank_score'].tolist())
    betweenness_label = np.array(betweenness_centrality_result_prc_all['label'].tolist())
    betweenness_precision_original, betweenness_recall_original, betweenness_thres = precision_recall_curve(betweenness_label, betweenness_score)
    betweenness_auc_precision_recall_original = auc(betweenness_recall_original, betweenness_precision_original)
    betweenness_auc_precision_recall_round_original=round(betweenness_auc_precision_recall_original,2)
    betweenness_false_positive_rate_original, betweenness_true_positive_rate_original, betweenness_threshold_original = roc_curve(betweenness_label, betweenness_score)
    betweenness_roc_original = roc_auc_score(betweenness_label, betweenness_score)
    betweenness_roc_original=round(betweenness_roc_original,2)
    return(betweenness_precision_original,
           betweenness_recall_original,
           betweenness_auc_precision_recall_round_original,
           betweenness_false_positive_rate_original,
           betweenness_true_positive_rate_original,
           betweenness_roc_original)

other_core_method=pd.read_csv('/h/tianyi/TS_datasets_reversion/primary/predict_drug/network_distance_CFX/close_distance_disease_drug_zscore_auc_DGIdb_cutoff15_L1000_all_screen_library/drug_network_distance_other_method_all.txt',sep='\t')
network_distance_other_core_method=pd.read_csv('/h/tianyi/TS_datasets_reversion/primary/predict_drug/network_distance_CFX/close_distance_auc_DGIdb_cutoff15_screen_library_plot/original_distance_add_other_zscore.txt',sep='\t')
network_distance_other_core_method_skin_braf=network_distance_other_core_method[(network_distance_other_core_method['tissue']=='Skin') & (network_distance_other_core_method['seed']=='BRAF')]
network_distance_other_core_method_skin_braf_drug=network_distance_other_core_method_skin_braf[network_distance_other_core_method_skin_braf['drugbank_id'].isin(cell_viability_distance['drugbank_id'])]
cell_viability_distance_1um_process=cell_viability_distance_1um
core_method_drug_values=network_distance_other_core_method_skin_braf_drug
centre_measure_prc_roc=prc_roc_core_method_value(network_distance_other_core_method_skin_braf_drug,'centre_value')
method_name='centre_value'
def prc_roc_core_method_value(core_method_drug_values,method_name):
    cell_viability_distance_1um_process=cell_viability_distance_1um.drop(['closest_distance_score'],axis=1)
    core_method_drug_values_hsa=pd.merge(core_method_drug_values,cell_viability_distance_1um_process,on='drugbank_id').dropna()
    core_method_drug_values_hsa_rename=core_method_drug_values_hsa.rename(columns={f'{method_name}':'closest_distance_score'})
    ######prc######
    pcr_data_all=network_distance_rank_label_prc(core_method_drug_values_hsa_rename)
    score = np.array(pcr_data_all['rank_score'].tolist())
    label = np.array(pcr_data_all['label'].tolist())
    precision_original, recall_original, thres = precision_recall_curve(label, score)
    auc_precision_recall_original = auc(recall_original, precision_original)
    auc_precision_recall_round_original=round(auc_precision_recall_original,2)
    ########roc#######
    score = np.array(pcr_data_all['rank_score'].tolist())
    label = np.array(pcr_data_all['label'].tolist())
    false_positive_rate_original, true_positive_rate_original, threshold_original = roc_curve(label, score)
    roc_original = roc_auc_score(label, score)
    roc_original=round(roc_original,2)
    prc_output=pd.DataFrame()
    prc_output['precision_original']=precision_original
    prc_output['recall_original']=recall_original
    prc_output['PRC_AUC']=auc_precision_recall_round_original
    roc_output=pd.DataFrame()
    roc_output['false_positive_rate_original']=false_positive_rate_original
    roc_output['true_positive_rate_original']=true_positive_rate_original
    roc_output['ROC_AUC']=roc_original
    roc_output['method']=method_name
    prc_roc_output=pd.concat([prc_output,roc_output],axis=1)
    return(prc_roc_output)
def prc_roc_core_method_value(core_method_drug_values,method_name):
    cell_viability_distance_1um_process=cell_viability_distance_1um.drop(['closest_distance_score'],axis=1)
    core_method_drug_values_hsa=pd.merge(core_method_drug_values,cell_viability_distance_1um_process,on='drugbank_id').dropna()
    core_method_drug_values_hsa_rename=core_method_drug_values_hsa.rename(columns={f'{method_name}':'closest_distance_score'})
    ######prc######
    pcr_data_all=network_distance_rank_label_prc(core_method_drug_values_hsa_rename)
    score = np.array(pcr_data_all['rank_score'].tolist())
    label = np.array(pcr_data_all['label'].tolist())
    precision_original, recall_original, thres = precision_recall_curve(label, score)
    auc_precision_recall_original = auc(recall_original, precision_original)
    auc_precision_recall_round_original=round(auc_precision_recall_original,2)
    ########roc#######
    score = np.array(pcr_data_all['rank_score'].tolist())
    label = np.array(pcr_data_all['label'].tolist())
    false_positive_rate_original, true_positive_rate_original, threshold_original = roc_curve(label, score)
    roc_original = roc_auc_score(label, score)
    roc_original=round(roc_original,2)
    prc_output=pd.DataFrame()
    prc_output['precision_original']=precision_original
    prc_output['recall_original']=recall_original
    prc_output['PRC_AUC']=auc_precision_recall_round_original
    roc_output=pd.DataFrame()
    roc_output['false_positive_rate_original']=false_positive_rate_original
    roc_output['true_positive_rate_original']=true_positive_rate_original
    roc_output['ROC_AUC']=roc_original
    roc_output['method']=method_name
    prc_roc_output=pd.concat([prc_output,roc_output],axis=1)
    return(prc_roc_output)
def other_method_rank_label_prc(top_gene_drug_top_num_label_sort,cell_viability_distance_1um_hsascore):
    top_gene_drug_top_num_label_sort_hsascore=pd.merge(top_gene_drug_top_num_label_sort,cell_viability_distance_1um_hsascore,left_on='DrugBank',right_on='drugbank_id',how='right')
    top_gene_drug_top_num_label_sort_hsascore['label']=1
    top_gene_drug_top_num_label_sort_hsascore.loc[top_gene_drug_top_num_label_sort_hsascore['HSA_score']<0.15,'label']=0
    top_gene_drug_top_num_label_sort_hsascore.fillna(0,inplace=True)
    top_gene_drug_top_num_label_sort_hsascore_score_zero=top_gene_drug_top_num_label_sort_hsascore[top_gene_drug_top_num_label_sort_hsascore['method_score']==0]
    top_gene_drug_top_num_label_sort_hsascore_score_zero_dgidb=pd.merge(top_gene_drug_top_num_label_sort_hsascore_score_zero,dgidb_score_max,left_on='drugbank_id',right_on='DrugBank')
    top_gene_drug_top_num_label_sort_hsascore_score_zero_dgidb_sort=top_gene_drug_top_num_label_sort_hsascore_score_zero_dgidb.sort_values('dgidb_score')
    top_gene_drug_top_num_label_sort_hsascore_score_zero_dgidb_sort.drop(['DrugBank_y','dgidb_score'],axis=1,inplace=True)
    top_gene_drug_top_num_label_sort_hsascore_score_zero_dgidb_sort.rename(columns={'DrugBank_x':'DrugBank'},inplace=True)
    top_gene_drug_top_num_label_sort_hsascore_score_nozero=top_gene_drug_top_num_label_sort_hsascore[top_gene_drug_top_num_label_sort_hsascore['method_score']!=0]
    top_gene_drug_top_num_label_sort_hsascore_score_nozero_sort=top_gene_drug_top_num_label_sort_hsascore_score_nozero.sort_values('method_score')
    top_gene_drug_top_num_label_sort_hsascore_score_sort=pd.concat([top_gene_drug_top_num_label_sort_hsascore_score_zero_dgidb_sort,top_gene_drug_top_num_label_sort_hsascore_score_nozero_sort])
    return(top_gene_drug_top_num_label_sort_hsascore_score_sort)
def other_method_plot_data_v3(betweenness_centrality_result):
    betweenness_centrality_result_drug_rank=core_convert_drug(betweenness_centrality_result)
    betweenness_centrality_result_prc_all=other_method_rank_label_prc(betweenness_centrality_result_drug_rank,cell_viability_distance_1um).drop_duplicates(['drugbank_id'])
    betweenness_centrality_result_prc_all.reset_index(drop=True,inplace=True)
    betweenness_centrality_result_prc_all['rank_score']=list(range(1,betweenness_centrality_result_prc_all.shape[0]+1))
    betweenness_score = np.array(betweenness_centrality_result_prc_all['rank_score'].tolist())
    betweenness_label = np.array(betweenness_centrality_result_prc_all['label'].tolist())
    betweenness_precision_original, betweenness_recall_original, betweenness_thres = precision_recall_curve(betweenness_label, betweenness_score)
    betweenness_auc_precision_recall_original = auc(betweenness_recall_original, betweenness_precision_original)
    betweenness_auc_precision_recall_round_original=round(betweenness_auc_precision_recall_original,2)
    betweenness_false_positive_rate_original, betweenness_true_positive_rate_original, betweenness_threshold_original = roc_curve(betweenness_label, betweenness_score)
    betweenness_roc_original = roc_auc_score(betweenness_label, betweenness_score)
    betweenness_roc_original=round(betweenness_roc_original,2)
    return(betweenness_precision_original,
           betweenness_recall_original,
           betweenness_auc_precision_recall_round_original,
           betweenness_false_positive_rate_original,
           betweenness_true_positive_rate_original,
           betweenness_roc_original)
def plot_correlation_v2(cell_viability_distance,drug_concentration):
os.chdir('/h/tianyi/TS_datasets_reversion/primary/predict_drug/different_methods_core_nodes_auc/Primary_OncoNiche/Skin')
network_distance_skin_braf_high=cell_viability_distance[cell_viability_distance['closest_distance_score']<=1.45]
slope_ori_comb, intercept_ori_comb, r_value_ori_comb, p_value_ori_comb, std_err_ori_comb = linregress(cell_viability_distance['closest_distance_score'].astype(float), cell_viability_distance['comb_drug_viability'].astype(float))
slope_ori_hsa, intercept_ori_hsa, r_value_ori_hsa, p_value_ori_hsa, std_err_ori_hsa = linregress(cell_viability_distance['closest_distance_score'].astype(float), cell_viability_distance['HSA_score'].astype(float))

betweenness_centrality_result=pd.read_csv('Skin_BRAF_betweenness_centrality_result.txt',sep='\t')
pagerank_result=pd.read_csv('Skin_BRAF_pagerank_result.txt',sep='\t')
closeness_centrality_result=pd.read_csv('Skin_BRAF_closeness_centrality_result.txt',sep='\t')
eccentricity_centrality_result=pd.read_csv('Skin_BRAF_eccentricity_centrality_result.txt',sep='\t')
degree_centrality_result=pd.read_csv('Skin_BRAF_degree_centrality_result.txt',sep='\t')
k_shell_result=pd.read_csv('Skin_BRAF_k_shell_result.txt',sep='\t')

betweenness_centrality_result_plot_data=other_method_plot_data_v3(betweenness_centrality_result)
pagerank_result_plot_data=other_method_plot_data_v3(pagerank_result)
closeness_centrality_result_plot_data=other_method_plot_data_v3(closeness_centrality_result)
eccentricity_centrality_result_plot_data=other_method_plot_data_v3(eccentricity_centrality_result)
degree_centrality_result_plot_data=other_method_plot_data_v3(degree_centrality_result)
k_shell_result_plot_data=other_method_plot_data_v3(k_shell_result)

######core method prc and roc #####

centre_measure_prc_roc=prc_roc_core_method_value(network_distance_other_core_method_skin_braf_drug,'centre_value')
shortest_measure_prc_roc=prc_roc_core_method_value(network_distance_other_core_method_skin_braf_drug,'shortest_value')
kernel_measure_prc_roc=prc_roc_core_method_value(network_distance_other_core_method_skin_braf_drug,'kernel_value')
network_proximity_z_score_prc_roc=prc_roc_core_method_value(network_distance_other_core_method_skin_braf_drug,'distance_z_score')

######prc######
pcr_data_all=network_distance_rank_label_prc(cell_viability_distance_1um)
score = np.array(pcr_data_all['rank_score'].tolist())
label = np.array(pcr_data_all['label'].tolist())
precision_original, recall_original, thres = precision_recall_curve(label, score)
auc_precision_recall_original = auc(recall_original, precision_original)
auc_precision_recall_round_original=round(auc_precision_recall_original,2)
########roc#######
score = np.array(pcr_data_all['rank_score'].tolist())
label = np.array(pcr_data_all['label'].tolist())
false_positive_rate_original, true_positive_rate_original, threshold_original = roc_curve(label, score)
roc_original = roc_auc_score(label, score)
roc_original=round(roc_original,2)

p_value_ori_comb='{:.2e}'.format(p_value_ori_comb)
r_value_ori_comb='{:.2e}'.format(r_value_ori_comb)


fig, axes = plt.subplots(2,1,figsize=(2.5*(67.778/48.993),2.5*2*(67.778/48.993)))
plt.rcParams['axes.linewidth'] = 0.5
#####drug comb cell viability######
axes[0].plot(recall_original, precision_original,color="#EE7B6C", linewidth=1)
axes[0].plot(centre_measure_prc_roc['recall_original'], centre_measure_prc_roc['precision_original'],color="#7BC7B3", linewidth=0.5)
axes[0].plot(shortest_measure_prc_roc['recall_original'], shortest_measure_prc_roc['precision_original'],color="#80AACA", linewidth=0.5)
axes[0].plot(kernel_measure_prc_roc['recall_original'], kernel_measure_prc_roc['precision_original'],color="#F6CC79", linewidth=0.5)
axes[0].plot(network_proximity_z_score_prc_roc['recall_original'], network_proximity_z_score_prc_roc['precision_original'],color="#A2A2A2", linewidth=0.5)
axes[0].plot(betweenness_centrality_result_plot_data[1], betweenness_centrality_result_plot_data[0],color="#EF9BBB", linewidth=0.5)
axes[0].plot(pagerank_result_plot_data[1], pagerank_result_plot_data[0],color="#7FC165", linewidth=0.5)
axes[0].plot(closeness_centrality_result_plot_data[1], closeness_centrality_result_plot_data[0],color='#CFE6ED', linewidth=0.5)
axes[0].plot(eccentricity_centrality_result_plot_data[1], eccentricity_centrality_result_plot_data[0],color='#9395C9', linewidth=0.5)
axes[0].plot(degree_centrality_result_plot_data[1], degree_centrality_result_plot_data[0],color='#7C4E23', linewidth=0.5)
axes[0].plot(k_shell_result_plot_data[1], k_shell_result_plot_data[0],color='#C775AC', linewidth=0.5)

axes[0].set_xlabel('Recall',fontsize=8, fontname='Arial')
axes[0].set_ylabel('Precision',fontsize=8, fontname='Arial')
axes[0].tick_params(axis='x',labelsize=8, width=0.5)
axes[0].tick_params(axis='y',labelsize=8, width=0.5)
axes[0].set_xlim(-0.1,1.1)
axes[0].set_ylim(-0.1,1.1)
axes[1].plot(false_positive_rate_original, true_positive_rate_original,color="#EE7B6C", linewidth=1)
axes[1].plot(centre_measure_prc_roc['false_positive_rate_original'], centre_measure_prc_roc['true_positive_rate_original'],color="#7BC7B3", linewidth=0.5)
axes[1].plot(shortest_measure_prc_roc['false_positive_rate_original'], shortest_measure_prc_roc['true_positive_rate_original'],color="#80AACA", linewidth=0.5)
axes[1].plot(kernel_measure_prc_roc['false_positive_rate_original'], kernel_measure_prc_roc['true_positive_rate_original'],color="#F6CC79", linewidth=0.5)
axes[1].plot(network_proximity_z_score_prc_roc['false_positive_rate_original'], network_proximity_z_score_prc_roc['true_positive_rate_original'],color="#A2A2A2", linewidth=0.5)
axes[1].plot(betweenness_centrality_result_plot_data[3], betweenness_centrality_result_plot_data[4],color="#EF9BBB", linewidth=0.5)
axes[1].plot(pagerank_result_plot_data[3], pagerank_result_plot_data[4],color="#7FC165", linewidth=0.5)
axes[1].plot(closeness_centrality_result_plot_data[3], closeness_centrality_result_plot_data[4],color="#CFE6ED", linewidth=0.5)
axes[1].plot(eccentricity_centrality_result_plot_data[3], eccentricity_centrality_result_plot_data[4],color="#9395C9", linewidth=0.5)
axes[1].plot(degree_centrality_result_plot_data[3], degree_centrality_result_plot_data[4],color="#7C4E23", linewidth=0.5)
axes[1].plot(k_shell_result_plot_data[3], k_shell_result_plot_data[4],color="#C775AC", linewidth=0.5)

axes[1].set_xlabel('False positive rate',fontsize=8, fontname='Arial')
axes[1].set_ylabel('True positive rate',fontsize=8, fontname='Arial')
axes[1].tick_params(axis='x',labelsize=8, width=0.5)
axes[1].tick_params(axis='y',labelsize=8, width=0.5)
axes[1].set_xlim(-0.1,1.1)
axes[1].set_ylim(-0.1,1.1)

plt.savefig(f'/h/tianyi/TS_datasets_reversion/graduate_plot/result/drug_combination_cell_viability_corelation_network_distance/cell_viability_hsa_distance_correlation_{drug_concentration}_add_prc_auc_figsize_v9.pdf', dpi=300, bbox_inches='tight')



In [ ]:
## cell viability and hsa distance correlation

cell_viability_distance_1um=pd.read_csv('/h/tianyi/TS_datasets_reversion/primary/predict_drug/high_throughput_drug_screening/CCK8_OD_1um_process/all_drug_score/cell_viability_distance_v5.txt',sep='\t')
cell_viability_distance=cell_viability_distance_1um
drug_concentration='1μm'
def network_distance_rank_label_prc(cell_viability_distance):
    top_gene_drug_top_num_label_sort=cell_viability_distance.sort_values('closest_distance_score')[::-1]
    top_gene_drug_top_num_label_sort['rank_score']=np.arange(1,top_gene_drug_top_num_label_sort.shape[0]+1)
    top_gene_drug_top_num_label_sort['label']=1
    top_gene_drug_top_num_label_sort.loc[top_gene_drug_top_num_label_sort['HSA_score']<0.15,'label']=0
    return(top_gene_drug_top_num_label_sort)
    
def plot_correlation_v2(cell_viability_distance,drug_concentration):
network_distance_skin_braf_high=cell_viability_distance[cell_viability_distance['closest_distance_score']<=1.45]
slope_ori_comb, intercept_ori_comb, r_value_ori_comb, p_value_ori_comb, std_err_ori_comb = linregress(cell_viability_distance['closest_distance_score'].astype(float), cell_viability_distance['comb_drug_viability'].astype(float))
slope_ori_hsa, intercept_ori_hsa, r_value_ori_hsa, p_value_ori_hsa, std_err_ori_hsa = linregress(cell_viability_distance['closest_distance_score'].astype(float), cell_viability_distance['HSA_score'].astype(float))

pcr_data_all=network_distance_rank_label_prc(cell_viability_distance_1um)

######prc######
score = np.array(pcr_data_all['rank_score'].tolist())
label = np.array(pcr_data_all['label'].tolist())
precision_original, recall_original, thres = precision_recall_curve(label, score)
auc_precision_recall_original = auc(recall_original, precision_original)
auc_precision_recall_round_original=round(auc_precision_recall_original,2)


########roc#######

score = np.array(pcr_data_all['rank_score'].tolist())
label = np.array(pcr_data_all['label'].tolist())
false_positive_rate_original, true_positive_rate_original, threshold_original = roc_curve(label, score)
roc_original = roc_auc_score(label, score)
roc_original=round(roc_original,2)

p_value_ori_comb='{:.2e}'.format(p_value_ori_comb)
r_value_ori_comb='{:.2e}'.format(r_value_ori_comb)
fig, axes = plt.subplots(1,1,figsize=(2.5,2.5))
plt.rcParams['axes.linewidth'] = 0.5
#####drug comb cell viability######
sns.scatterplot(y=cell_viability_distance['HSA_score'].astype(float),x=cell_viability_distance['closest_distance_score'],ax=axes,color=(143/255,134/255,132/255),s=18)
sns.scatterplot(y=network_distance_skin_braf_high['HSA_score'].astype(float),x=network_distance_skin_braf_high['closest_distance_score'],ax=axes,color='#EE7B6C',s=18)
sns.lineplot(cell_viability_distance['closest_distance_score'].astype(float), slope_ori_hsa * cell_viability_distance['closest_distance_score'].astype(float) + intercept_ori_hsa, color='#80AACA',ax=axes,linewidth=0.5)
axes.text(3.3,-0.15,f'r = -0.31\n', fontsize=8,fontname='Arial')
axes.text(3.3,-0.18,f'p < 0.001\n', fontsize=8,fontname='Arial')
axes.axvline(x=1.48, color='#EE7B6C', linestyle='--', linewidth=0.5)
axes.axhline(y=0.15, color='#EE7B6C', linestyle='--', linewidth=0.5)
axes.set_ylabel(f'HSA score in drug combination ({drug_concentration})',fontsize=8, fontname='Arial')
axes.set_xlabel('Network proximity',fontsize=8, fontname='Arial')
axes.tick_params(axis='x', labelsize=8, width=0.5)
axes.tick_params(axis='y', labelsize=8, width=0.5)
plt.savefig(f'/h/tianyi/TS_datasets_reversion/sci_paper_plot_stas/data_statistic/cell_viability_hsa_distance_correlation/cell_viability_hsa_distance_correlation_{drug_concentration}_v2.pdf', dpi=300, bbox_inches='tight')


#########drug comb cell viability  confidence######

library(ggplot2) 
setwd('G:\\课题\\tissue_specificity_manuscript\\素材\\cell_viability_hsa_distance_correlation')
cell_viability_distance_1um=data.frame(read.table('cell_viability_distance_v5.txt',sep='\t',header=1))
#regression model 
fit <- lm(HSA_score ~ closest_distance_score, data = cell_viability_distance_1um) 
ci <- predict(fit, interval = "confidence", 
              level = 0.95) 

cell_viability_distance_1um$plot_color ='#8F8684'
cell_viability_distance_1um[which(cell_viability_distance_1um$closest_distance_score <1.45),]$plot_color='#EE7B6C'

(0.5/1.07)*0.5
p_point=ggplot(data = cell_viability_distance_1um, aes(x = closest_distance_score, y = HSA_score)) + 
  geom_point(size = 1.497, color = "#8F8684", data = ~subset(., plot_color == "#8F8684")) + 
  geom_point(size = 1.497, color = "#EE7B6C", data = ~subset(., plot_color == "#EE7B6C")) + 
  geom_smooth(method = "lm", se = FALSE) + 
  geom_ribbon(aes(ymin = ci[, 2], ymax = ci[, 3]), 
              alpha = 0.2,size=(0.5/1.07)*0.5) +   
  geom_vline(xintercept = 1.45, color = "#EE7B6C", size = (0.5/1.07)*0.5, linetype = "dashed") + # 添加垂直线
  geom_hline(yintercept = 0.15, color = "#EE7B6C", size = (0.5/1.07)*0.5, linetype = "dashed") + # 添加水平线
  ylab('HSA score in drug combination (1μm)') + 
  xlab('Network proximity')+
  theme(
    panel.border = element_blank(),
    axis.line.y = element_line(size = (0.5/1.07) * 0.5),
    axis.line.x = element_line(size = (0.5/1.07) * 0.5),
    axis.ticks.y = element_line(size = (0.5/1.07) * 0.5),
    axis.ticks.x = element_line(size = (0.5/1.07) * 0.5),
    panel.grid.major = element_blank(),
    panel.grid.minor = element_blank(),
    axis.line = element_line(colour = "black"),
    panel.background = element_rect(fill = "transparent", color = NA, size = (0.5/1.07) * 0.5),
    axis.text = element_text(size = 8),
    axis.title = element_text(size = 8),
    legend.text = element_text(size = 8),
    legend.title = element_text(size = 8),
    plot.title = element_text(size = 8, hjust = 0.5),
    axis.text.x = element_text(size = 8, family = "sans", color = "black"),
    axis.text.y = element_text(size = 8, family = "sans", color = "black"),
  )

pdf('cell_viability_hsa_distance_correlation_1μm_R_v4.pdf',width=2.951272, height=2.951272)
p_point
dev.off()